In [ ]:
import pandas as pd
import numpy as np
import statistics
from statistics import mode
import math
from collections import Counter

file1=pd.read_csv("/content/seeds_dataset.txt",delim_whitespace=True,header=None)
print(file1)
x=file1.iloc[:,:-1].values
y=file1.iloc[:,-1].values
print(x)

         0      1       2      3      4      5      6  7
0    15.26  14.84  0.8710  5.763  3.312  2.221  5.220  1
1    14.88  14.57  0.8811  5.554  3.333  1.018  4.956  1
2    14.29  14.09  0.9050  5.291  3.337  2.699  4.825  1
3    13.84  13.94  0.8955  5.324  3.379  2.259  4.805  1
4    16.14  14.99  0.9034  5.658  3.562  1.355  5.175  1
..     ...    ...     ...    ...    ...    ...    ... ..
205  12.19  13.20  0.8783  5.137  2.981  3.631  4.870  3
206  11.23  12.88  0.8511  5.140  2.795  4.325  5.003  3
207  13.20  13.66  0.8883  5.236  3.232  8.315  5.056  3
208  11.84  13.21  0.8521  5.175  2.836  3.598  5.044  3
209  12.30  13.34  0.8684  5.243  2.974  5.637  5.063  3

[210 rows x 8 columns]
[[15.26   14.84    0.871  ...  3.312   2.221   5.22  ]
 [14.88   14.57    0.8811 ...  3.333   1.018   4.956 ]
 [14.29   14.09    0.905  ...  3.337   2.699   4.825 ]
 ...
 [13.2    13.66    0.8883 ...  3.232   8.315   5.056 ]
 [11.84   13.21    0.8521 ...  2.836   3.598   5.044 ]
 [12.3    13

In [ ]:
for i in range(0,len(x)):
  for j in range(0,len(x[0])):
    x[i][j]=float(x[i][j])
print(x)


[[15.26   14.84    0.871  ...  3.312   2.221   5.22  ]
 [14.88   14.57    0.8811 ...  3.333   1.018   4.956 ]
 [14.29   14.09    0.905  ...  3.337   2.699   4.825 ]
 ...
 [13.2    13.66    0.8883 ...  3.232   8.315   5.056 ]
 [11.84   13.21    0.8521 ...  2.836   3.598   5.044 ]
 [12.3    13.34    0.8684 ...  2.974   5.637   5.063 ]]


In [ ]:
def find_avgcluster_matrix(matrix_x):
  if np.min(matrix_x)==100000.0:
    print("no more cluster possible")
  else:
    cluster=np.where(matrix_x==np.min(matrix_x))
    cluster=np.squeeze(cluster)
    cluster=np.sort(cluster)
    ar1=list(matrix_x[cluster[0]])
    ar2=list(matrix_x[cluster[1]])
    ac1=[item[cluster[0]] for item in matrix_x]
    ac2=[item[cluster[1]] for item in matrix_x]
    for i in range(0,len(ar1)):
      if ar1[i]!=100000.0 and ar2[i]!=100000.0:
        mini=[]
        mini.append(ar1[i])
        mini.append(ar2[i])
        ar1[i]=(mini[0]+mini[1])/2
    for i in range(0,len(ar1)):
      if ac1[i]!=100000.0 and ar2[i]!=100000.0:
        mini=[]
        mini.append(ac1[i])
        mini.append(ar2[i])
        ac1[i]=(mini[0]+mini[1])/2
    for i in range(0,len(ar1)):
      if ac1[i]!=100000.0 and ac2[i]!=100000.0:
        mini=[]
        mini.append(ac1[i])
        mini.append(ac2[i])
        ac1[i]=(mini[0]+mini[1])/2
    for i in range(0,len(ar1)):
      matrix_x[cluster[1],i]=100000.0
    for i in range(0,len(ac1)):
      matrix_x[i,cluster[1]]=100000.0
    kp=ac1[cluster[1]]
    for i in range(0,len(ac1)):
      if ac1[i]==kp:
        ac1[i]=100000.0
    for i in range(0,len(ar1)):
      matrix_x[cluster[0],i]=ar1[i]
    for i in range(0,len(ac1)):
      matrix_x[i,cluster[0]]=ac1[i]
    return matrix_x,cluster;

In [ ]:

def find_avgcluster(x,clustno):
  clust=[]
  k=len(x)
  matrix_x=np.zeros((k,k))
  for i in range(0,len(matrix_x)):
    for j in range(0,len(matrix_x)):
      if i>j:
        matrix_x[i,j]=float(np.linalg.norm(np.array(x[i])-np.array(x[j])))
      else:
        matrix_x[i,j]=100000
  for po in range (0,len(matrix_x)-1):
    matrix_x,cluster=find_avgcluster_matrix(matrix_x)
    cluster=list(cluster)
    k1=0
    for item in clust:
      for k in range(0,len(item)):
        if item[k]==cluster[0]:
          item.append(cluster[1])
          k1=1
          break
        if item[k]==cluster[1]:
          item.append(cluster[0])
          k1=1
          break
    if k1==0:
      clust.append(cluster)
    merge=[]
    for item in clust:
      pi=0
      for k in range(0,len(item)):
        if item[k]==cluster[0] or item[k]==cluster[1]:
          pi=pi+1
      if pi==2:
        merge.append(clust.index(item))
    if len(merge)==2:
      clust[merge[0]]=clust[merge[0]]+clust[merge[1]]
      clust.remove(clust[merge[1]])
    for i in range (0,len(clust)):
      uniq1=clust[i]
      uniq1=np.asarray(uniq1)
      uniq1=np.unique(uniq1)
      clust[i]=list(uniq1)
    clno=len(matrix_x)-po-1
    if clustno==clno:
      clusti2=[]
      for item in clust:
        for i in range(0,len(item)):
          clusti2.append(item[i])
      for i in range(0,len(x)):
        k=0
        for j in range(0,len(clusti2)):
          if i==clusti2[j]:
            k=1
        if k==0:
          clust.append([i])
      return clust



In [ ]:
def find_avgcluster_matrix(matrix_x):
  if np.min(matrix_x)==100000.0:
    print("no more cluster possible")
  else:
    cluster=np.where(matrix_x==np.min(matrix_x))
    cluster=np.squeeze(cluster)
    cluster=np.sort(cluster)
    ar1=list(matrix_x[cluster[0]])
    ar2=list(matrix_x[cluster[1]])
    ac1=[item[cluster[0]] for item in matrix_x]
    ac2=[item[cluster[1]] for item in matrix_x]
    for i in range(0,len(ar1)):
      if ar1[i]!=100000.0 and ar2[i]!=100000.0:
        mini=[]
        mini.append(ar1[i])
        mini.append(ar2[i])
        ar1[i]=(mini[0]+mini[1])/2
    for i in range(0,len(ar1)):
      if ac1[i]!=100000.0 and ar2[i]!=100000.0:
        mini=[]
        mini.append(ac1[i])
        mini.append(ar2[i])
        ac1[i]=(mini[0]+mini[1])/2
    for i in range(0,len(ar1)):
      if ac1[i]!=100000.0 and ac2[i]!=100000.0:
        mini=[]
        mini.append(ac1[i])
        mini.append(ac2[i])
        ac1[i]=(mini[0]+mini[1])/2
    for i in range(0,len(ar1)):
      matrix_x[cluster[1],i]=100000.0
    for i in range(0,len(ac1)):
      matrix_x[i,cluster[1]]=100000.0
    kp=ac1[cluster[1]]
    for i in range(0,len(ac1)):
      if ac1[i]==kp:
        ac1[i]=100000.0
    for i in range(0,len(ar1)):
      matrix_x[cluster[0],i]=ar1[i]
    for i in range(0,len(ac1)):
      matrix_x[i,cluster[0]]=ac1[i]
    return matrix_x,cluster;

In [ ]:

def find_avgcluster(x,clustno):
  clust=[]
  k=len(x)
  matrix_x=np.zeros((k,k))
  for i in range(0,len(matrix_x)):
    for j in range(0,len(matrix_x)):
      if i>j:
        matrix_x[i,j]=float(np.linalg.norm(np.array(x[i])-np.array(x[j])))
      else:
        matrix_x[i,j]=100000
  for po in range (0,len(matrix_x)-1):
    matrix_x,cluster=find_avgcluster_matrix(matrix_x)
    cluster=list(cluster)
    k1=0
    for item in clust:
      for k in range(0,len(item)):
        if item[k]==cluster[0]:
          item.append(cluster[1])
          k1=1
          break
        if item[k]==cluster[1]:
          item.append(cluster[0])
          k1=1
          break
    if k1==0:
      clust.append(cluster)
    merge=[]
    for item in clust:
      pi=0
      for k in range(0,len(item)):
        if item[k]==cluster[0] or item[k]==cluster[1]:
          pi=pi+1
      if pi==2:
        merge.append(clust.index(item))
    if len(merge)==2:
      clust[merge[0]]=clust[merge[0]]+clust[merge[1]]
      clust.remove(clust[merge[1]])
    for i in range (0,len(clust)):
      uniq1=clust[i]
      uniq1=np.asarray(uniq1)
      uniq1=np.unique(uniq1)
      clust[i]=list(uniq1)
    clno=len(matrix_x)-po-1
    if clustno==clno:
      clusti2=[]
      for item in clust:
        for i in range(0,len(item)):
          clusti2.append(item[i])
      for i in range(0,len(x)):
        k=0
        for j in range(0,len(clusti2)):
          if i==clusti2[j]:
            k=1
        if k==0:
          clust.append([i])
      return clust



In [ ]:
def predict1(intt,clustfea,testin,y,k):
  dist=[]
  test1=list(intt[testin])
  testclust=clustfea[testin]
  for i in range(0,len(intt)):
    ki1=list(intt[i])
    ki2=clustfea[i]
    distance=0
    for j in range(0,len(intt[0])):
      point1=np.square(float(ki1[j])-float(test1[j]))
      distance=distance+point1
    for j in range(0,len(clustfea[0])):
      point1=np.square(float(ki2[j])-float(testclust[j]))
      distance=distance+point1
    distance=np.sqrt(distance)
    dist.append(distance)
  dist_sort=sorted(dist)
  k1=dist_sort[:k]
  neigh=[]
  for i in k1:
    for j in dist:
      if i==j:
        neigh.append(dist.index(i))
  p=[]
  for i in neigh:
    p.append(y[i])
  return mode(p)
  

In [ ]:
def predict2(intt,clustfea,test1,testclust,y,k):
  dist=[]
  for i in range(0,len(intt)):
    ki1=list(intt[i])
    ki2=clustfea[i]
    distance=0
    for j in range(0,len(intt[0])):
      point1=np.square(float(ki1[j])-float(test1[j]))
      distance=distance+point1
    for j in range(0,len(clustfea[0])):
      point1=np.square(float(ki2[j])-float(testclust[j]))
      distance=distance+point1
    distance=np.sqrt(distance)
    dist.append(distance)
  dist_sort=sorted(dist)
  k1=dist_sort[:k]
  neigh=[]
  for i in k1:
    for j in dist:
      if i==j:
        neigh.append(dist.index(i))
  p=[]
  for i in neigh:
    p.append(y[i])
  return mode(p)
  

In [ ]:
def predict(intt,test1,y,k):
  dist=[]
  for i in range(0,len(intt)):
    ki1=list(intt[i])
    distance=0
    for j in range(0,len(intt[0])):
      point1=np.square(float(ki1[j])-float(test1[j]))
      distance=distance+point1
    distance=np.sqrt(distance)
    dist.append(distance)
  dist_sort=sorted(dist)
  k1=dist_sort[:k]
  neigh=[]
  for i in k1:
    for j in dist:
      if i==j:
        neigh.append(dist.index(i))
  p=[]
  for i in neigh:
    p.append(y[i])
  modep1=findmultimode(p)
  return modep1
  

In [101]:
def findknn(ki):
  split1=len(file1)-30
  x=file1.iloc[:split1,:-1].values
  y=file1.iloc[:split1,-1].values
  x_test=file1.iloc[split1:,:-1].values
  y_test=file1.iloc[split1:,-1].values
  print("actual y without using cluster")
  print(y_test)
  yclust5=[]
  for i in range(0,len(x_test)):
    kp1=predict(x,list(x_test[i]),y,ki)
    yclust5.append(kp1)
  yclust5=np.asarray(yclust5)
  print("predicted y without using cluster")
  print(yclust5)
  count1=0
  for i in range(0,len(y_test)):
    if y_test[i]==yclust5[i]:
      count1=count1+1
  accuracy1=float(count1)/float(len(y_test))
  print("accuracy for without using cluster") 
  print(accuracy1)
  print(" ")


In [ ]:
def findmultimode(p):
  modep=[]
  p1=Counter(p)
  q1=p1.most_common(1)[0][1]
  for item in p1:
    if p.count(item)==q1:
      modep.append(item)
  modep=list(set(modep))
  return modep[0]

In [ ]:
def findclusteraccuracyavg(clustno):
  file1=pd.read_csv("/content/seeds_dataset.txt",delim_whitespace=True,header=None)
  y=file1.iloc[:,-1].values
  clust=find_avgcluster(x,clustno)
  clustp=file1.iloc[:,-1].values
  clusted=[]
  for i in range(0,len(clust)):
    clust1=[]
    for j in range(0,len(clust[i])):
      clust1.append(y[clust[i][j]])
    clusted.append(clust1)
  vote=[]
  for i in range(0,len(clusted)):
    kg1=findmultimode(clusted[i])
    vote.append(kg1)
  for i in range(0,len(clust)):
    for j in range(0,len(clust[i])):
      clustp[clust[i][j]]=vote[i]

  count1=0
  file1=pd.read_csv("/content/seeds_dataset.txt",delim_whitespace=True,header=None)
  y=file1.iloc[:,-1].values

  for i in range(0,len(y)):
    if y[i]==clustp[i]:
      count1=count1+1
  accuracy1=float(count1)/float(len(y))
  return accuracy1

In [ ]:
def findbestclusteravg(x):
  accu=[]
  for i in range(1,len(x)):
    accu.append(findclusteraccuracyavg(i))
  for i in range(0,len(accu)):
    if accu[i]>=0.9:
      return i+1
      break

In [ ]:
def findaccuracyavglink(ki1):
  x=file1.iloc[:,:-1].values
  y=file1.iloc[:,-1].values
  clustno=findbestclusteravg(x)
  print("a good cluster is")
  print(clustno)
  clust5=find_avgcluster(x,clustno)
  avgclustpoint=[]
  for i in range(0,len(clust5)):
    clustp=[]
    for j in range(0,len(x[0])):
      a12=0
      for k in range(0,len(clust5[i])):
        kp=x[clust5[i][k]]
        kp1=kp[j]
        a12=a12+kp1
      a12=a12/len(clust5[i])
      clustp.append(a12)
    avgclustpoint.append(clustp)
  clustfeature=[]
  for i in range(0,len(avgclustpoint)):
    cluster1=[]
    for j in range(0,len(x)):
      datapoint=x[j]
      distance=0
      clustpoint=avgclustpoint[i]
      for k in range(0,len(datapoint)):
        point1=np.square(float(clustpoint[k])-float(datapoint[k]))
        distance=distance+point1
      distance=np.sqrt(distance)
      cluster1.append(distance)
    clustfeature.append(cluster1)
  clustfeature1=[]
  for i in range(0,len(x)):
    clust123=[]
    for j in range(0,len(clustfeature)):
      uv=clustfeature[j][i]
      clust123.append(uv)
    clustfeature1.append(clust123)
  split1=len(file1)-30
  clustfeaturetrain1=clustfeature1[:split1]
  clustfeaturetest1=clustfeature1[split1:]
  x=file1.iloc[:split1,:-1].values
  y=file1.iloc[:split1,-1].values
  x_test=file1.iloc[split1:,:-1].values
  y_test=file1.iloc[split1:,-1].values
  yclust5=[]
  for i in range(0,len(x_test)):
    kp1=predict2(x,clustfeaturetrain1,list(x_test[i]),clustfeaturetest1[i],y,ki1)
    yclust5.append(kp1)
  yclust5=np.asarray(yclust5)
  print("actutal y")
  print(y_test)
  print("predicted y")
  print(yclust5)
  count1=0
  for i in range(0,len(y_test)):
    if y_test[i]==yclust5[i]:
      count1=count1+1
  accuracy1=float(count1)/float(len(y_test))
  print("accuracy for "+str(clustno)+" cluster") 
  print(accuracy1)
  print(" ")

In [103]:
findknn(3)
findknn(5)
findknn(7)

actual y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 3 3 1 3 3 3 3]
accuracy for without using cluster
0.9
 
actual y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 1 3 3 3 1 3 3 3 3]
accuracy for without using cluster
0.8666666666666667
 
actual y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 3 1 3 3 3 3 3 3 3 3]
accuracy for without using cluster
0.8666666666666667
 


In [ ]:
findaccuracyavglink(3)
findaccuracyavglink(5)
findaccuracyavglink(7)

a good cluster is
18
actutal y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 1 3 3 3 3 3 3 3 3]
accuracy for 18 cluster
0.9
 
a good cluster is
18
actutal y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 3 1 3 3 3 3 3 3 3 3]
accuracy for 18 cluster
0.8666666666666667
 
a good cluster is
18
actutal y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 3 1 3 3 3 3 3 3 3 3]
accuracy for 18 cluster
0.8666666666666667
 


In [ ]:
def findaccuracyavglink1(ki1):
  x=file1.iloc[:,:-1].values
  y=file1.iloc[:,-1].values
  clustno=findbestclusteravg(x)
  print("a good cluster is")
  print(clustno)
  clust5=find_avgcluster(x,clustno)
  print(clust5)
  avgclustpoint=[]
  for i in range(0,len(clust5)):
    clustp=[]
    for j in range(0,len(x[0])):
      a12=0
      for k in range(0,len(clust5[i])):
        kp=x[clust5[i][k]]
        kp1=kp[j]
        a12=a12+kp1
      a12=a12/len(clust5[i])
      clustp.append(a12)
    avgclustpoint.append(clustp)
  clustfeature=[]
  for i in range(0,len(avgclustpoint)):
    cluster1=[]
    for j in range(0,len(x)):
      datapoint=x[j]
      distance=0
      clustpoint=avgclustpoint[i]
      for k in range(0,len(datapoint)):
        point1=np.square(float(clustpoint[k])-float(datapoint[k]))
        distance=distance+point1
      distance=np.sqrt(distance)
      cluster1.append(distance)
    clustfeature.append(cluster1)
  clustfeature1=[]
  for i in range(0,len(x)):
    clust123=[]
    for j in range(0,len(clustfeature)):
      uv=clustfeature[j][i]
      clust123.append(uv)
    clustfeature1.append(clust123)
  yclust5=[]
  for i in range(0,len(x)):
    kp1=predict1(x,clustfeature1,i,y,ki1)
    yclust5.append(kp1)
  yclust5=np.asarray(yclust5)
  print("actutal y")
  print(y)
  print("predicted y")
  print(yclust5)
  count1=0
  for i in range(0,len(y)):
    if y[i]==yclust5[i]:
      count1=count1+1
  accuracy1=float(count1)/float(len(y))
  print("accuracy for "+str(clustno)+" cluster") 
  print(accuracy1)
  print(" ")

In [ ]:
findaccuracyavglink1(3)
findaccuracyavglink1(5)
findaccuracyavglink1(7)

a good cluster is
18
[[19, 144, 147, 149, 150, 152, 153, 154, 156, 157, 158, 159, 161, 162, 163, 166, 167, 168, 169, 172, 173, 174, 175, 176, 177, 180, 181, 182, 183, 185, 186, 187, 189, 190, 191, 192, 193, 194, 202, 204, 205, 206, 208], [12, 13, 14, 26, 27, 29, 42, 62, 63, 65, 69, 148, 160, 197, 198, 199, 201], [70, 74, 76, 95, 100, 107, 121, 122, 133, 136, 139], [2, 5, 6, 7, 15, 20, 21, 28, 31, 32, 38, 44, 47, 48, 50, 52, 53, 56, 124, 135], [8, 9, 35, 36, 37, 79, 137, 138], [0, 1, 4, 17, 18, 22, 24, 25, 34, 46, 49, 54, 55, 57, 58, 66, 68], [3, 11, 23, 30, 33, 40, 41, 45, 59, 64, 67, 179], [16, 140, 142, 143, 184, 195, 196, 200, 209], [73, 78, 84, 87, 91, 92, 94, 96, 102, 103, 104, 106, 108, 111, 116, 117, 118, 119, 125, 126, 131], [85, 86, 97, 98, 99, 101, 105, 109, 110, 112, 115, 123, 127, 130], [10, 43, 51, 71, 72, 75, 80, 129, 132, 134], [83, 90, 128], [145, 151, 155, 164, 170, 171, 178, 188], [77, 82, 88, 89, 114, 120], [60, 61, 146, 165], [203, 207], [81, 93, 113], [39, 141]]
ac

In [ ]:
def find_mincluster_matrix(matrix_x):
  if np.min(matrix_x)==100000.0:
    print("no more cluster possible")
  else:
    cluster=np.where(matrix_x==np.min(matrix_x))
    cluster=np.squeeze(cluster)
    cluster=np.sort(cluster)
    ar1=list(matrix_x[cluster[0]])
    ar2=list(matrix_x[cluster[1]])
    ac1=[item[cluster[0]] for item in matrix_x]
    ac2=[item[cluster[1]] for item in matrix_x]
    for i in range(0,len(ar1)):
      if ar1[i]!=100000.0 and ar2[i]!=100000.0:
        mini=[]
        mini.append(ar1[i])
        mini.append(ar2[i])
        ar1[i]=min(mini)
    for i in range(0,len(ar1)):
      if ac1[i]!=100000.0 and ar2[i]!=100000.0:
        mini=[]
        mini.append(ac1[i])
        mini.append(ar2[i])
        ac1[i]=min(mini)
    for i in range(0,len(ar1)):
      if ac1[i]!=100000.0 and ac2[i]!=100000.0:
        mini=[]
        mini.append(ac1[i])
        mini.append(ac2[i])
        ac1[i]=min(mini)
    for i in range(0,len(ar1)):
      matrix_x[cluster[1],i]=100000.0
    for i in range(0,len(ac1)):
      matrix_x[i,cluster[1]]=100000.0
    kp=ac1[cluster[1]]
    for i in range(0,len(ac1)):
      if ac1[i]==kp:
        ac1[i]=100000.0
    for i in range(0,len(ar1)):
      matrix_x[cluster[0],i]=ar1[i]
    for i in range(0,len(ac1)):
      matrix_x[i,cluster[0]]=ac1[i]
    return matrix_x,cluster;

In [ ]:
def find_mincluster(x,clustno):
  clust=[]
  k=len(x)
  matrix_x=np.zeros((k,k))
  for i in range(0,len(matrix_x)):
    for j in range(0,len(matrix_x)):
      if i>j:
        matrix_x[i,j]=float(np.linalg.norm(np.array(x[i])-np.array(x[j])))
      else:
        matrix_x[i,j]=100000
  for po in range (0,len(matrix_x)-1):
    matrix_x,cluster=find_mincluster_matrix(matrix_x)
    cluster=list(cluster)
    k1=0
    for item in clust:
      for k in range(0,len(item)):
        if item[k]==cluster[0]:
          item.append(cluster[1])
          k1=1
          break
        if item[k]==cluster[1]:
          item.append(cluster[0])
          k1=1
          break
    if k1==0:
      clust.append(cluster)
    merge=[]
    for item in clust:
      pi=0
      for k in range(0,len(item)):
        if item[k]==cluster[0] or item[k]==cluster[1]:
          pi=pi+1
      if pi==2:
        merge.append(clust.index(item))
    if len(merge)==2:
      clust[merge[0]]=clust[merge[0]]+clust[merge[1]]
      clust.remove(clust[merge[1]])
    for i in range (0,len(clust)):
      uniq1=clust[i]
      uniq1=np.asarray(uniq1)
      uniq1=np.unique(uniq1)
      clust[i]=list(uniq1)
    clno=len(matrix_x)-po-1
    if clustno==clno:
      clusti2=[]
      for item in clust:
        for i in range(0,len(item)):
          clusti2.append(item[i])
      for i in range(0,len(x)):
        k=0
        for j in range(0,len(clusti2)):
          if i==clusti2[j]:
            k=1
        if k==0:
          clust.append([i])
      return clust



In [ ]:
def findclusteraccuracymin(clustno):
  file1=pd.read_csv("/content/seeds_dataset.txt",delim_whitespace=True,header=None)
  y=file1.iloc[:,-1].values
  clust=find_mincluster(x,clustno)
  clustp=file1.iloc[:,-1].values
  clusted=[]
  for i in range(0,len(clust)):
    clust1=[]
    for j in range(0,len(clust[i])):
      clust1.append(y[clust[i][j]])
    clusted.append(clust1)
  vote=[]
  for i in range(0,len(clusted)):
    kg1=findmultimode(clusted[i])
    vote.append(kg1)
  for i in range(0,len(clust)):
    for j in range(0,len(clust[i])):
      clustp[clust[i][j]]=vote[i]

  count1=0
  file1=pd.read_csv("/content/seeds_dataset.txt",delim_whitespace=True,header=None)
  y=file1.iloc[:,-1].values

  for i in range(0,len(y)):
    if y[i]==clustp[i]:
      count1=count1+1
  accuracy1=float(count1)/float(len(y))
  return accuracy1

In [ ]:
def findbestclustermin(x):
  accu=[]
  for i in range(1,len(x)):
    accu.append(findclusteraccuracymin(i))
  for i in range(0,len(accu)):
    if accu[i]>=0.9:
      return i+1
      break

In [ ]:
def findaccuracyminlink(ki1):
  x=file1.iloc[:,:-1].values
  y=file1.iloc[:,-1].values
  clustno=findbestclustermin(x)
  print("a good cluster is")
  print(clustno)
  clust5=find_mincluster(x,clustno)
  avgclustpoint=[]
  for i in range(0,len(clust5)):
    clustp=[]
    for j in range(0,len(x[0])):
      a12=0
      for k in range(0,len(clust5[i])):
        kp=x[clust5[i][k]]
        kp1=kp[j]
        a12=a12+kp1
      a12=a12/len(clust5[i])
      clustp.append(a12)
    avgclustpoint.append(clustp)
  clustfeature=[]
  for i in range(0,len(avgclustpoint)):
    cluster1=[]
    for j in range(0,len(x)):
      datapoint=x[j]
      distance=0
      clustpoint=avgclustpoint[i]
      for k in range(0,len(datapoint)):
        point1=np.square(float(clustpoint[k])-float(datapoint[k]))
        distance=distance+point1
      distance=np.sqrt(distance)
      cluster1.append(distance)
    clustfeature.append(cluster1)
  clustfeature1=[]
  for i in range(0,len(x)):
    clust123=[]
    for j in range(0,len(clustfeature)):
      uv=clustfeature[j][i]
      clust123.append(uv)
    clustfeature1.append(clust123)
  split1=len(file1)-30
  clustfeaturetrain1=clustfeature1[:split1]
  clustfeaturetest1=clustfeature1[split1:]
  x=file1.iloc[:split1,:-1].values
  y=file1.iloc[:split1,-1].values
  x_test=file1.iloc[split1:,:-1].values
  y_test=file1.iloc[split1:,-1].values
  yclust5=[]
  for i in range(0,len(x_test)):
    kp1=predict2(x,clustfeaturetrain1,list(x_test[i]),clustfeaturetest1[i],y,ki1)
    yclust5.append(kp1)
  yclust5=np.asarray(yclust5)
  print("actutal y")
  print(y_test)
  print("predicted y")
  print(yclust5)
  count1=0
  for i in range(0,len(y_test)):
    if y_test[i]==yclust5[i]:
      count1=count1+1
  accuracy1=float(count1)/float(len(y_test))
  print("accuracy for "+str(clustno)+" cluster") 
  print(accuracy1)
  print(" ")

In [102]:
findknn(3)
findknn(5)
findknn(7)


actual y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 3 3 1 3 3 3 3]
accuracy for without using cluster
0.9
 
actual y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 1 3 3 3 1 3 3 3 3]
accuracy for without using cluster
0.8666666666666667
 
actual y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 3 1 3 3 3 3 3 3 3 3]
accuracy for without using cluster
0.8666666666666667
 


In [ ]:
findaccuracyminlink(3)
findaccuracyminlink(5)
findaccuracyminlink(7)

a good cluster is
46
actutal y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 1 3 3 3 3 3 3 3 3]
accuracy for 46 cluster
0.9
 
a good cluster is
46
actutal y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 1 3 3 3 3 3 3 3 3]
accuracy for 46 cluster
0.9
 
a good cluster is
46
actutal y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 3 1 3 3 3 3 3 3 3 3]
accuracy for 46 cluster
0.8666666666666667
 


In [ ]:
def findaccuracyminlink1(ki1):
  x=file1.iloc[:,:-1].values
  y=file1.iloc[:,-1].values
  clustno=findbestclustermin(x)
  print("a good cluster is")
  print(clustno)
  clust5=find_mincluster(x,clustno)
  print(clust5)
  avgclustpoint=[]
  for i in range(0,len(clust5)):
    clustp=[]
    for j in range(0,len(x[0])):
      a12=0
      for k in range(0,len(clust5[i])):
        kp=x[clust5[i][k]]
        kp1=kp[j]
        a12=a12+kp1
      a12=a12/len(clust5[i])
      clustp.append(a12)
    avgclustpoint.append(clustp)
  clustfeature=[]
  for i in range(0,len(avgclustpoint)):
    cluster1=[]
    for j in range(0,len(x)):
      datapoint=x[j]
      distance=0
      clustpoint=avgclustpoint[i]
      for k in range(0,len(datapoint)):
        point1=np.square(float(clustpoint[k])-float(datapoint[k]))
        distance=distance+point1
      distance=np.sqrt(distance)
      cluster1.append(distance)
    clustfeature.append(cluster1)
  clustfeature1=[]
  for i in range(0,len(x)):
    clust123=[]
    for j in range(0,len(clustfeature)):
      uv=clustfeature[j][i]
      clust123.append(uv)
    clustfeature1.append(clust123)
  yclust5=[]
  for i in range(0,len(x)):
    kp1=predict1(x,clustfeature1,i,y,ki1)
    yclust5.append(kp1)
  yclust5=np.asarray(yclust5)
  print("actutal y")
  print(y)
  print("predicted y")
  print(yclust5)
  count1=0
  for i in range(0,len(y)):
    if y[i]==yclust5[i]:
      count1=count1+1
  accuracy1=float(count1)/float(len(y))
  print("accuracy for "+str(clustno)+" cluster") 
  print(accuracy1)
  print(" ")

In [ ]:
findaccuracyminlink1(3)
findaccuracyminlink1(5)
findaccuracyminlink1(7)

a good cluster is
46
[[19, 62, 63, 143, 144, 145, 147, 149, 150, 152, 153, 154, 155, 156, 157, 158, 159, 161, 162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 180, 181, 182, 183, 185, 186, 187, 189, 190, 191, 192, 193, 194, 195, 196, 197, 200, 202, 204, 205, 206, 208, 209], [2, 3, 5, 6, 7, 11, 13, 14, 15, 20, 21, 26, 28, 29, 31, 32, 33, 38, 40, 41, 44, 45, 47, 48, 50, 52, 53, 54, 56, 66, 67, 68, 69, 135, 148, 160, 198], [100, 122, 133, 139], [137, 138], [0, 4, 17, 22, 24, 25, 34, 46, 49, 55, 58], [73, 84, 85, 86, 87, 91, 92, 96, 97, 98, 99, 101, 103, 104, 105, 106, 108, 109, 110, 111, 112, 115, 116, 117, 118, 119, 123, 125, 126, 127, 130, 131], [71, 72, 75], [90, 128], [1, 57], [27, 42, 65, 199, 201], [37, 79], [70, 74, 76, 95, 107, 121, 136], [10, 43, 132, 134], [77, 89, 114], [8, 9, 35], [23, 59, 179], [142, 184], [60, 61], [203, 207], [12], [16], [18], [30], [36], [39], [51], [64], [78], [80], [81], [82], [83], [88], [93], [94], [102], [113], [120], [1

In [ ]:
def find_maxcluster_matrix(matrix_x):
  if np.min(matrix_x)==100000.0:
    print("no more cluster possible")
  else:
    cluster=np.where(matrix_x==np.min(matrix_x))
    cluster=np.squeeze(cluster)
    cluster=np.sort(cluster)
    ar1=list(matrix_x[cluster[0]])
    ar2=list(matrix_x[cluster[1]])
    ac1=[item[cluster[0]] for item in matrix_x]
    ac2=[item[cluster[1]] for item in matrix_x]
    for i in range(0,len(ar1)):
      if ar1[i]!=100000.0 and ar2[i]!=100000.0:
        mini=[]
        mini.append(ar1[i])
        mini.append(ar2[i])
        ar1[i]=max(mini)
    for i in range(0,len(ar1)):
      if ac1[i]!=100000.0 and ar2[i]!=100000.0:
        mini=[]
        mini.append(ac1[i])
        mini.append(ar2[i])
        ac1[i]=max(mini)
    for i in range(0,len(ar1)):
      if ac1[i]!=100000.0 and ac2[i]!=100000.0:
        mini=[]
        mini.append(ac1[i])
        mini.append(ac2[i])
        ac1[i]=max(mini)
    for i in range(0,len(ar1)):
      matrix_x[cluster[1],i]=100000.0
    for i in range(0,len(ac1)):
      matrix_x[i,cluster[1]]=100000.0
    kp=ac1[cluster[1]]
    for i in range(0,len(ac1)):
      if ac1[i]==kp:
        ac1[i]=100000.0
    for i in range(0,len(ar1)):
      matrix_x[cluster[0],i]=ar1[i]
    for i in range(0,len(ac1)):
      matrix_x[i,cluster[0]]=ac1[i]
    return matrix_x,cluster;

In [ ]:
def find_maxcluster(x,clustno):
  clust=[]
  k=len(x)
  matrix_x=np.zeros((k,k))
  for i in range(0,len(matrix_x)):
    for j in range(0,len(matrix_x)):
      if i>j:
        matrix_x[i,j]=float(np.linalg.norm(np.array(x[i])-np.array(x[j])))
      else:
        matrix_x[i,j]=100000
  for po in range (0,len(matrix_x)-1):
    matrix_x,cluster=find_maxcluster_matrix(matrix_x)
    cluster=list(cluster)
    k1=0
    for item in clust:
      for k in range(0,len(item)):
        if item[k]==cluster[0]:
          item.append(cluster[1])
          k1=1
          break
        if item[k]==cluster[1]:
          item.append(cluster[0])
          k1=1
          break
    if k1==0:
      clust.append(cluster)
    merge=[]
    for item in clust:
      pi=0
      for k in range(0,len(item)):
        if item[k]==cluster[0] or item[k]==cluster[1]:
          pi=pi+1
      if pi==2:
        merge.append(clust.index(item))
    if len(merge)==2:
      clust[merge[0]]=clust[merge[0]]+clust[merge[1]]
      clust.remove(clust[merge[1]])
    for i in range (0,len(clust)):
      uniq1=clust[i]
      uniq1=np.asarray(uniq1)
      uniq1=np.unique(uniq1)
      clust[i]=list(uniq1)
    clno=len(matrix_x)-po-1
    if clustno==clno:
      clusti2=[]
      for item in clust:
        for i in range(0,len(item)):
          clusti2.append(item[i])
      for i in range(0,len(x)):
        k=0
        for j in range(0,len(clusti2)):
          if i==clusti2[j]:
            k=1
        if k==0:
          clust.append([i])
      return clust



In [ ]:
def findclusteraccuracymax(clustno):
  file1=pd.read_csv("/content/seeds_dataset.txt",delim_whitespace=True,header=None)
  y=file1.iloc[:,-1].values
  clust=find_maxcluster(x,clustno)
  clustp=file1.iloc[:,-1].values
  clusted=[]
  for i in range(0,len(clust)):
    clust1=[]
    for j in range(0,len(clust[i])):
      clust1.append(y[clust[i][j]])
    clusted.append(clust1)
  vote=[]
  for i in range(0,len(clusted)):
    kg1=findmultimode(clusted[i])
    vote.append(kg1)
  for i in range(0,len(clust)):
    for j in range(0,len(clust[i])):
      clustp[clust[i][j]]=vote[i]

  count1=0
  file1=pd.read_csv("/content/seeds_dataset.txt",delim_whitespace=True,header=None)
  y=file1.iloc[:,-1].values

  for i in range(0,len(y)):
    if y[i]==clustp[i]:
      count1=count1+1
  accuracy1=float(count1)/float(len(y))
  return accuracy1

In [ ]:
def findbestclustermax(x):
  accu=[]
  for i in range(1,len(x)):
    accu.append(findclusteraccuracymax(i))
  for i in range(0,len(accu)):
    if accu[i]>=0.9:
      return i+1
      break

In [ ]:
def findaccuracymaxlink(ki1):
  x=file1.iloc[:,:-1].values
  y=file1.iloc[:,-1].values
  clustno=findbestclustermax(x)
  print("a good cluster is")
  print(clustno)
  clust5=find_maxcluster(x,clustno)
  avgclustpoint=[]
  for i in range(0,len(clust5)):
    clustp=[]
    for j in range(0,len(x[0])):
      a12=0
      for k in range(0,len(clust5[i])):
        kp=x[clust5[i][k]]
        kp1=kp[j]
        a12=a12+kp1
      a12=a12/len(clust5[i])
      clustp.append(a12)
    avgclustpoint.append(clustp)
  clustfeature=[]
  for i in range(0,len(avgclustpoint)):
    cluster1=[]
    for j in range(0,len(x)):
      datapoint=x[j]
      distance=0
      clustpoint=avgclustpoint[i]
      for k in range(0,len(datapoint)):
        point1=np.square(float(clustpoint[k])-float(datapoint[k]))
        distance=distance+point1
      distance=np.sqrt(distance)
      cluster1.append(distance)
    clustfeature.append(cluster1)
  clustfeature1=[]
  for i in range(0,len(x)):
    clust123=[]
    for j in range(0,len(clustfeature)):
      uv=clustfeature[j][i]
      clust123.append(uv)
    clustfeature1.append(clust123)
  split1=len(file1)-30
  clustfeaturetrain1=clustfeature1[:split1]
  clustfeaturetest1=clustfeature1[split1:]
  x=file1.iloc[:split1,:-1].values
  y=file1.iloc[:split1,-1].values
  x_test=file1.iloc[split1:,:-1].values
  y_test=file1.iloc[split1:,-1].values
  yclust5=[]
  for i in range(0,len(x_test)):
    kp1=predict2(x,clustfeaturetrain1,list(x_test[i]),clustfeaturetest1[i],y,ki1)
    yclust5.append(kp1)
  yclust5=np.asarray(yclust5)
  print("actutal y")
  print(y_test)
  print("predicted y")
  print(yclust5)
  count1=0
  for i in range(0,len(y_test)):
    if y_test[i]==yclust5[i]:
      count1=count1+1
  accuracy1=float(count1)/float(len(y_test))
  print("accuracy for "+str(clustno)+" cluster") 
  print(accuracy1)
  print(" ")

In [104]:
findknn(3)
findknn(5)
findknn(7)

actual y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 3 3 1 3 3 3 3]
accuracy for without using cluster
0.9
 
actual y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 1 3 3 3 1 3 3 3 3]
accuracy for without using cluster
0.8666666666666667
 
actual y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y without using cluster
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 3 1 3 3 3 3 3 3 3 3]
accuracy for without using cluster
0.8666666666666667
 


In [ ]:
findaccuracymaxlink(3)
findaccuracymaxlink(5)
findaccuracymaxlink(7)

a good cluster is
13
actutal y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 3 1 3 1 3 3 3 3 3 3 3 3]
accuracy for 13 cluster
0.9
 
a good cluster is
13
actutal y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 3 1 3 3 3 3 3 3 3 3]
accuracy for 13 cluster
0.8666666666666667
 
a good cluster is
13
actutal y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
predicted y
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 1 1 1 3 1 3 3 3 3 3 3 3 3]
accuracy for 13 cluster
0.8666666666666667
 


In [ ]:
def findaccuracymaxlink1(ki1):
  x=file1.iloc[:,:-1].values
  y=file1.iloc[:,-1].values
  clustno=findbestclustermax(x)
  print("a good cluster is")
  print(clustno)
  clust5=find_maxcluster(x,clustno)
  print(clust5)
  avgclustpoint=[]
  for i in range(0,len(clust5)):
    clustp=[]
    for j in range(0,len(x[0])):
      a12=0
      for k in range(0,len(clust5[i])):
        kp=x[clust5[i][k]]
        kp1=kp[j]
        a12=a12+kp1
      a12=a12/len(clust5[i])
      clustp.append(a12)
    avgclustpoint.append(clustp)
  clustfeature=[]
  for i in range(0,len(avgclustpoint)):
    cluster1=[]
    for j in range(0,len(x)):
      datapoint=x[j]
      distance=0
      clustpoint=avgclustpoint[i]
      for k in range(0,len(datapoint)):
        point1=np.square(float(clustpoint[k])-float(datapoint[k]))
        distance=distance+point1
      distance=np.sqrt(distance)
      cluster1.append(distance)
    clustfeature.append(cluster1)
  clustfeature1=[]
  for i in range(0,len(x)):
    clust123=[]
    for j in range(0,len(clustfeature)):
      uv=clustfeature[j][i]
      clust123.append(uv)
    clustfeature1.append(clust123)
  yclust5=[]
  for i in range(0,len(x)):
    kp1=predict1(x,clustfeature1,i,y,ki1)
    yclust5.append(kp1)
  yclust5=np.asarray(yclust5)
  print("actutal y")
  print(y)
  print("predicted y")
  print(yclust5)
  count1=0
  for i in range(0,len(y)):
    if y[i]==yclust5[i]:
      count1=count1+1
  accuracy1=float(count1)/float(len(y))
  print("accuracy for "+str(clustno)+" cluster") 
  print(accuracy1)
  print(" ")

In [ ]:
findaccuracymaxlink1(3)
findaccuracymaxlink1(5)
findaccuracymaxlink1(7)

a good cluster is
13
[[149, 150, 153, 154, 156, 159, 168, 169, 172, 173, 174, 175, 176, 177, 180, 183, 185, 187, 189, 190, 191, 193, 202, 206], [26, 60, 61, 62, 69, 146, 148, 160, 165, 167, 192, 198, 205, 208], [8, 9, 31, 35, 36, 37, 74, 79, 95, 100, 122, 124, 133, 135, 137, 138, 139], [2, 5, 6, 7, 12, 13, 14, 15, 20, 21, 28, 29, 32, 38, 44, 47, 48, 50, 52, 53, 56], [0, 1, 3, 4, 11, 17, 18, 22, 24, 25, 33, 34, 45, 46, 49, 54, 55, 57, 58, 66, 67, 68], [16, 19, 63, 140, 143, 144, 147, 152, 157, 158, 161, 162, 163, 166, 181, 182, 186, 194, 195, 196, 197, 200, 204, 209], [73, 83, 84, 85, 86, 87, 90, 91, 92, 96, 97, 98, 99, 101, 102, 103, 104, 105, 106, 108, 109, 110, 111, 112, 115, 116, 117, 118, 119, 123, 125, 126, 127, 128, 130, 131], [70, 71, 72, 75, 76, 107, 121, 129, 136], [23, 27, 30, 40, 41, 42, 59, 64, 65, 179, 199, 201], [145, 151, 155, 164, 170, 171, 178, 188], [10, 43, 51, 80, 132, 134], [77, 78, 81, 82, 88, 89, 93, 94, 113, 114, 120], [39, 141, 142, 184, 203, 207]]
actutal y
[1